In [10]:
import string
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [11]:
# from seqlearn.perceptron import StructuredPerceptron

# clf = StructuredPerceptron()

In [12]:
# def features(sequence, i):
#     yield "word=" + sequence[i].lower()
#     if sequence[i].isupper():
#         yield "Uppercase"

In [13]:
# from seqlearn.datasets import load_conll
# X_train, y_train, lenths_train = load_conll("train.txt", features)

In [14]:
def prepare_for_RNN(X_train, Y_train, X_test, Y_test):
    
    zero_array = [0]*len(X_train[0][0])
    padding_X_train = []
    for i, sentence in enumerate(X_train):
        for j, token in enumerate(sentence):
            temp_point = []
            if j==0 or j==len(sentence)-1:
                temp_point.append(zero_array)
                temp_point.append(zero_array)
                temp_point.append(sentence[j])
                padding_X_train.append(temp_point)
            elif j==1 or j==len(sentence)-2:
                temp_point.append(zero_array)
                temp_point.append(sentence[j-1])
                temp_point.append(sentence[j])
                padding_X_train.append(temp_point)
            else:
                temp_point.append(sentence[j-2])
                temp_point.append(sentence[j-1])
                temp_point.append(sentence[j])
                padding_X_train.append(temp_point)    
    padding_X_train = np.array(padding_X_train, dtype='float32')
    unfolded_Y_train = []
    for sentence in Y_train:
        for token in sentence:
            unfolded_Y_train.append(token)
    encoder = LabelEncoder()
    encoder.fit(unfolded_Y_train)
    encoded_Y = encoder.transform(unfolded_Y_train)
    dummy_y = np_utils.to_categorical(encoded_Y)
    
    ###############################################################
    zero_array = [0]*len(X_test[0][0])
    padding_X_test = []
    for i, sentence in enumerate(X_test):
        for j, token in enumerate(sentence):
            temp_point = []
            if j==0 or j==len(sentence)-1:
                temp_point.append(zero_array)
                temp_point.append(zero_array)
                temp_point.append(sentence[j])
                padding_X_test.append(temp_point)
            elif j==1 or j==len(sentence)-2:
                temp_point.append(zero_array)
                temp_point.append(sentence[j-1])
                temp_point.append(sentence[j])
                padding_X_test.append(temp_point)
            else:
                temp_point.append(sentence[j-2])
                temp_point.append(sentence[j-1])
                temp_point.append(sentence[j])
                padding_X_test.append(temp_point)    
    padding_X_test = np.array(padding_X_test, dtype='float32')
    unfolded_Y_test = []
    for sentence in Y_test:
        for token in sentence:
            unfolded_Y_test.append(token)
    encoder = LabelEncoder()
    encoder.fit(unfolded_Y_test)
    encoded_Y = encoder.transform(unfolded_Y_test)
    dummy_y_test = np_utils.to_categorical(encoded_Y)
    return padding_X_train, dummy_y, padding_X_test, dummy_y_test

In [15]:
# Choose what you want to use among various kinds of features
using_1type = 1
using_2type = 1
using_3type = 1
using_4type = 1

import numpy as np
import math
from pandas import read_csv 
from sklearn.model_selection import cross_val_score 
from sklearn.model_selection import KFold 
from sklearn.preprocessing import LabelEncoder 
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report 
from sklearn.metrics import precision_recall_fscore_support 
from sklearn.metrics import confusion_matrix 
from sklearn.metrics import accuracy_score 
from data_handler import *
from data_handling_for_NNs import *

# Large CNN model for the CIFAR-10 Dataset 
from keras.datasets import cifar10
from keras.models import Sequential 
from keras.wrappers.scikit_learn import KerasClassifier 
from keras.utils import np_utils 
from keras.layers import Dense 
from keras.layers import Dropout 
from keras.layers import Flatten 
from keras.constraints import maxnorm
from keras.optimizers import SGD 
from keras.layers.convolutional import Convolution2D 
from keras.layers.convolutional import MaxPooling2D 
from keras.utils import np_utils 
from keras import backend as K 
K.set_image_dim_ordering('th')

# CNN for the IMDB problem 
from keras.datasets import imdb 
from keras.layers.convolutional import Convolution1D 
from keras.layers.convolutional import MaxPooling1D 
from keras.layers.embeddings import Embedding 
from keras.preprocessing import sequence 

# LSTM with Variable Length Input Sequences to One Character Output 
from keras.layers import LSTM 
from keras.utils import np_utils 
from keras.preprocessing.sequence import pad_sequences

feature_path = 'assets/feature-vec/'
tempdata_path = 'assets/temp-data/'

### Data Load
FE_X_train = load(feature_path+'FE_X_train')
FE_X_test = load(feature_path+'FE_X_test')
################################### FE
FE2_X_train = load(feature_path+'FE2_X_train')
FE2_X_test = load(feature_path+'FE2_X_test')
################################### FE 2
FE3_X_train = load(feature_path+'FE3_X_train')
FE3_X_test = load(feature_path+'FE3_X_test')
################################### FE 3
FE4_X_train = load(feature_path+'FE4_X_train')
FE4_X_test = load(feature_path+'FE4_X_test')
Y_train = load(feature_path+'Ytrain')
Y_test = load(feature_path+'Ytest')
################################### Y
feature_info_without_LK = load('feature_info_without_LK')

### Choice Feature Set 
X_train, X_test = choice_feature_set(using_1type, using_2type, using_3type, using_4type,\
                                     FE_X_train, FE2_X_train, FE3_X_train, FE4_X_train,\
                                     FE_X_test, FE2_X_test, FE3_X_test, FE4_X_test)

print('===> Total feature length =', len(X_train[0][0]))

### Prepare data for each model
X_train_FNN, Y_train_FNN, X_test_FNN, Y_test_FNN = prepare_for_FNN(X_train, Y_train, X_test, Y_test)    
X_train_CNN, Y_train_CNN, X_test_CNN, Y_test_CNN = prepare_for_CNN(X_train, Y_train, X_test, Y_test)   
X_train_RNN, Y_train_RNN, X_test_RNN, Y_test_RNN = prepare_for_RNN(X_train, Y_train, X_test, Y_test)     

assert(len(X_train) == len(Y_train))
assert(len(X_test) == len(Y_test))
#assert(len(X_test[0]) == len(X_train[0]))
assert(len(X_train_FNN) == len(Y_train_FNN))
assert(len(X_test_FNN) == len(Y_test_FNN))
assert(len(X_train_CNN) == len(Y_train_CNN))
assert(len(X_test_CNN) == len(Y_test_CNN))
assert(len(X_train_RNN) == len(Y_train_RNN))
assert(len(X_test_RNN) == len(Y_test_RNN))

===> Total feature length = 10121


## Model Description

In [16]:
# for FNN
input_dim = len(X_train_FNN[0])
hidden_dim = 17
output_dim = len(Y_train_FNN[0])
n_epoch_fnn = 10

# for CNN
num_classes = Y_train_CNN.shape[1]
x_square = X_train_CNN.shape[3]
n_epoch_cnn = 3

# for RNN
time_length = X_train_RNN.shape[1]
feature_dim = X_train_RNN.shape[2]
n_hidden_node = 100
output_dim = Y_train_RNN.shape[1]
n_epoch_rnn = 3
batch_size = 1 

def feedforward():
    model = Sequential()
    model.add(Dense(hidden_dim, input_dim = input_dim, init='normal', activation='relu')) 
    model.add(Dense(output_dim, init='normal', activation='sigmoid'))
    # Compile model 
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])     
    return model

def convolutional():
    model = Sequential() 
    model.add(Convolution2D(32, 3, 3, input_shape=(1, x_square, x_square), activation='relu', border_mode='same')) 
#     model.add(Dropout(0.2)) 
    model.add(Convolution2D(32, 3, 3, activation='relu', border_mode='same')) 
    model.add(MaxPooling2D(pool_size=(2, 2)))
#     model.add(Convolution2D(64, 3, 3, activation='relu', border_mode='same')) 
#     model.add(Dropout(0.2)) 
#     model.add(Convolution2D(64, 3, 3, activation='relu', border_mode='same')) 
#     model.add(MaxPooling2D(pool_size=(2, 2))) 
    # model.add(Convolution2D(128, 3, 3, activation='relu', border_mode='same')) 
    # model.add(Dropout(0.2)) 
    # model.add(Convolution2D(128, 3, 3, activation='relu', border_mode='same')) 
    # model.add(MaxPooling2D(pool_size=(2, 2))) 
    model.add(Flatten()) 
    # model.add(Dropout(0.2)) 
    # model.add(Dense(1024, activation='relu', W_constraint=maxnorm(3))) 
    # model.add(Dropout(0.2)) 
    model.add(Dense(512, activation='relu', W_constraint=maxnorm(3))) 
    # model.add(Dropout(0.2)) 
    model.add(Dense(num_classes, activation='softmax')) 

    # Compile model 
    lrate = 0.1 
    decay = lrate/n_epoch_cnn 
    sgd = SGD(lr=lrate, momentum=0.9, decay=decay, nesterov=False) 
    model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy']) 
    #print(model.summary()) 
    return model

def recurrent():
    model = Sequential() 
    model.add(LSTM(n_hidden_node, input_shape=(time_length, feature_dim))) 
    model.add(Dense(output_dim, activation='softmax')) 
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy']) 
    #print(model.summary())  
    return model
    

## Model Training

In [17]:
print("*******************  Model Training  *********************")
#for name, model, X, Y in models:
#     kfold = KFold(n_splits=num_folds, random_state=seed)
#     cv_results = cross_val_score(model, unfolded_X_train, unfolded_Y_train, cv=kfold, scoring=scoring)
#     results.append(cv_results)
#     names.append(name)
#     msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
#     print(msg)

# just fit
print('>>> FNN Training...')
FNN_model = feedforward()
FNN_model.fit(X_train_FNN, Y_train_FNN, nb_epoch=n_epoch_fnn, batch_size=5, verbose=1)
print('\n')
print('>>> CNN Training...')
CNN_model = convolutional()
CNN_model.fit(X_train_CNN, Y_train_CNN, nb_epoch=n_epoch_cnn, batch_size=64, verbose=1) 
print('\n')
print('>>> RNN Training...')   
RNN_model = recurrent()
RNN_model.fit(X_train_RNN, Y_train_RNN, nb_epoch=n_epoch_rnn, batch_size=batch_size, verbose=1)


*******************  Model Training  *********************
>>> FNN Training...


C:\Users\Office\Anaconda3\lib\site-packages\ipykernel\__main__.py:22: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(17, activation="relu", kernel_initializer="normal", input_dim=10121)`
C:\Users\Office\Anaconda3\lib\site-packages\ipykernel\__main__.py:23: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(17, activation="sigmoid", kernel_initializer="normal")`
C:\Users\Office\Anaconda3\lib\site-packages\keras\models.py:874: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Epoch 1/10
10236/10236 [==============================] - 8s 760us/step - loss: 1.3913 - acc: 0.5491
Epoch 2/10
10236/10236 [==============================] - 7s 721us/step - loss: 0.7568 - acc: 0.7535
Epoch 3/10
10236/10236 [==============================] - 7s 725us/step - loss: 0.5317 - acc: 0.8302
Epoch 4/10
10236/10236 [==============================] - 7s 711us/step - loss: 0.3671 - acc: 0.8899
Epoch 5/10
10236/10236 [==============================] - 7s 710us/step - loss: 0.2443 - acc: 0.9301
Epoch 6/10
10236/10236 [==============================] - 7s 718us/step - loss: 0.1551 - acc: 0.9599
Epoch 7/10
10236/10236 [==============================] - 7s 711us/step - loss: 0.0968 - acc: 0.9777
Epoch 8/10
10236/10236 [==============================] - 7s 717us/step - loss: 0.0587 - acc: 0.9873
Epoch 9/10
10236/10236 [==============================] - 7s 710us/step - loss: 0.0362 - acc: 0.9938
Epoch 10/10
10236/10236 [==============================] - 7s 724us/step - loss: 0.0215 - a

C:\Users\Office\Anaconda3\lib\site-packages\ipykernel\__main__.py:30: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), activation="relu", padding="same", input_shape=(1, 101, 1...)`
C:\Users\Office\Anaconda3\lib\site-packages\ipykernel\__main__.py:32: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), activation="relu", padding="same")`
C:\Users\Office\Anaconda3\lib\site-packages\ipykernel\__main__.py:46: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(512, kernel_constraint=<keras.con..., activation="relu")`


Epoch 1/3
10236/10236 [==============================] - 559s 55ms/step - loss: 1.3846 - acc: 0.5582
Epoch 2/3
10236/10236 [==============================] - 566s 55ms/step - loss: 0.7645 - acc: 0.7440
Epoch 3/3
10236/10236 [==============================] - 562s 55ms/step - loss: 0.5606 - acc: 0.8114


>>> RNN Training...
Epoch 1/3
10236/10236 [==============================] - 1260s 123ms/step - loss: 0.7783 - acc: 0.7585
Epoch 2/3
10236/10236 [==============================] - 1224s 120ms/step - loss: 0.1647 - acc: 0.9495
Epoch 3/3
10236/10236 [==============================] - 1169s 114ms/step - loss: 0.0437 - acc: 0.9879


## Model Evaluation

In [18]:
def show_result(model, X_test, Y_test):
    predictions = model.predict_classes(X_test, verbose=0)
    print(accuracy_score(Y_test, predictions))
    print(confusion_matrix(Y_test, predictions))
    print('\n')
    print(classification_report(Y_test, predictions)) 
    
    pre, rec, f1, sup = (precision_recall_fscore_support(Y_test, predictions, average='micro'))
    print('micro => ', pre, '\t', rec, '\t', f1, '\t', sup)
    pre, rec, f1, sup = (precision_recall_fscore_support(Y_test, predictions, average='macro'))
    print('macro => ', pre, '\t', rec, '\t', f1, '\t', sup)
    pre, rec, f1, sup = (precision_recall_fscore_support(Y_test, predictions, average='weighted'))
    print('weighted => ',pre, '\t', rec, '\t', f1, '\t', sup)
    
    print('\n')

In [19]:
print("*******************  Model Evaluation  *********************")

print('>>> FNN Evaluation')
show_result(FNN_model, X_test_FNN, dummy_to_integer(Y_test_FNN))
#scores = model.evaluate(X_test_FNN, Y_test_FNN, verbose=0)
#print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    
print('\n')
print('>>> CNN1 Evaluation')
show_result(CNN_model, X_test_CNN, dummy_to_integer(Y_test_CNN))
#scores = model.evaluate(X_test_CNN, Y_test_CNN, verbose=0) 
#print("Accuracy: %.2f%%" % (scores[1]*100))

print('\n')
print('>>> RNN Evaluation')    
show_result(RNN_model, X_test_RNN, dummy_to_integer(Y_test_RNN))
#scores = model.evaluate(X_test_RNN, Y_test_RNN, verbose=1) 
#print("\nModel Accuracy: %.2f%%" % (scores[1]*100)) 

*******************  Model Evaluation  *********************
>>> FNN Evaluation
0.747356051704
[[ 73   0   0   0   1   1   0   1  12   0   0   0   0   0   0   0   4]
 [  1   7   1   3   0   0   0   0   4   5   0   0   0   0   0   0   9]
 [  1   0 133   6   0   1   0   0   5   4   2   0   0   0   2   0   9]
 [  1   3   1 219   5   1   0   0  22  56   0   3   0   0   2   4  20]
 [  2   0   2  20 108   0   2   0   2  16   0   1   0   0   0   1   8]
 [  0   0   4   1   0 198   0   0   1   1   0   0   0   0   0   0   6]
 [  0   0   3   7   4   0 109   0   1   6   1   0   0   1   3   0   9]
 [  1   0   0   0   0   0   0 136   0   0   0   0   0   0   0   0   5]
 [  6   0   1  27   3   1   2   0 208  16   1   0   0   0   2   0  14]
 [  0   1   0  41   3   0   2   0  13 148   1   9   1   0   4  13  13]
 [  0   0   4   1   0   0   1   0   1   0   8   0   0   0   1   0   2]
 [  0   0   0  12   1   0   0   0   0  13   0  25   0   0   0   1   1]
 [  0   0   0   0   0   0   0   0   0   0   0   0   7

In [20]:
# *******************  Model Evaluation  *********************
# >>> FNN Evaluation
# (2583,)
# 0.730158730159
# [[ 57   1   0   0   0   1   0   1   5   1   0   0   0   0   0   0   6]
#  [  0   5   0   5   0   0   0   0   1   5   0   0   0   0   0   0   4]
#  [  0   0 144   3   0   3   4   0   3   2   4   0   0   0   0   0   8]
#  [  0   0   1 187   9   0   3   0   6  54   0   8   0   0   3  12  25]
#  [  0   0   2  19  73   0  19   1   1  11   0   1   0   0   0   1   9]
#  [  2   0   3   0   0 230   2   0   0   0   0   0   1   0   0   0  13]
#  [  0   0   0   5  10   0 100   0   0   4   1   3   0   0   3   0   6]
#  [  5   0   0   0   0   0   0 144   0   0   0   0   0   0   0   0   2]
#  [  6   0   2  16   2   2   8   1 204   3   0   0   0   0   6   1  12]
#  [  1   6   4  59   2   1  16   0   6 135   2  13   1   0   0   3  33]
#  [  0   0   4   4   0   0   1   0   0   1   8   0   0   0   0   0   3]
#  [  0   0   0   7   0   0   1   0   2   5   0   4   0   0   0   0   6]
#  [  0   0   0   0   0   1   2   0   0   0   0   0   7   1   0   0   2]
#  [  0   0   0   0   0   0   0   3   0   1   0   0   0  10   0   0   0]
#  [  0   0   0   3   0   0   0   0   6   7   0   0   0   0  13   1   3]
#  [  0   0   0   7   0   0   0   0   3  38   0   4   0   0   0  38   5]
#  [  2   0   4  13   1   9   7   1  11  17   0   1   0   0   0   2 527]]


#              precision    recall  f1-score   support

#           0       0.78      0.79      0.79        72
#           1       0.42      0.25      0.31        20
#           2       0.88      0.84      0.86       171
#           3       0.57      0.61      0.59       308
#           4       0.75      0.53      0.62       137
#           5       0.93      0.92      0.92       251
#           6       0.61      0.76      0.68       132
#           7       0.95      0.95      0.95       151
#           8       0.82      0.78      0.80       263
#           9       0.48      0.48      0.48       282
#          10       0.53      0.38      0.44        21
#          11       0.12      0.16      0.14        25
#          12       0.78      0.54      0.64        13
#          13       0.91      0.71      0.80        14
#          14       0.52      0.39      0.45        33
#          15       0.66      0.40      0.50        95
#          16       0.79      0.89      0.84       595

# avg / total       0.73      0.73      0.73      2583



# >>> CNN1 Evaluation
# (2583,)
# 0.708091366628
# [[ 54   2   2   0   1   0   0   1   6   3   0   0   0   0   0   0   3]
#  [  0   6   0   3   0   1   0   0   0   9   0   0   0   0   0   0   1]
#  [  0   0 155   3   0   0   3   0   1   6   0   0   0   0   0   0   3]
#  [  0   1   4 138   6   0   4   0   6 124   0   9   0   0   0   0  16]
#  [  0   4   1  12  73   1  15   0   4  16   0   1   0   0   0   0  10]
#  [  2   0   7   0   1 229   0   0   0   0   0   0   0   0   0   0  12]
#  [  0   1   0  11  13   2  82   0   0  12   2   0   0   0   1   0   8]
#  [  5   0   0   0   0   0   0 144   0   0   0   0   0   0   0   0   2]
#  [  9   1  10   5   3   1   6   0 183  25   0   0   0   0   3   0  17]
#  [  1   7   6  46   1   0   3   0   2 197   0   2   0   0   0   0  17]
#  [  0   0   8   2   0   0   0   0   1   4   5   0   0   0   0   0   1]
#  [  0   0   0   8   1   0   0   0   0  11   0   2   0   0   0   0   3]
#  [  0   0   0   0   0   1   3   0   0   1   0   0   6   1   0   0   1]
#  [  0   0   0   0   0   0   2   2   0   2   0   0   0   8   0   0   0]
#  [  0   0   0   1   1   0   2   0   4  11   0   0   0   0  12   0   2]
#  [  0   0   0   6   0   0   2   0   0  74   0   0   0   0   0   6   7]
#  [  2   0   7  12   1  10   2   2   1  27   0   2   0   0   0   0 529]]


#              precision    recall  f1-score   support

#           0       0.74      0.75      0.74        72
#           1       0.27      0.30      0.29        20
#           2       0.78      0.91      0.84       171
#           3       0.56      0.45      0.50       308
#           4       0.72      0.53      0.61       137
#           5       0.93      0.91      0.92       251
#           6       0.66      0.62      0.64       132
#           7       0.97      0.95      0.96       151
#           8       0.88      0.70      0.78       263
#           9       0.38      0.70      0.49       282
#          10       0.71      0.24      0.36        21
#          11       0.12      0.08      0.10        25
#          12       1.00      0.46      0.63        13
#          13       0.89      0.57      0.70        14
#          14       0.75      0.36      0.49        33
#          15       1.00      0.06      0.12        95
#          16       0.84      0.89      0.86       595

# avg / total       0.75      0.71      0.70      2583



# >>> RNN Evaluation
# (2583,)
# 0.739450251645
# [[ 59   1   0   0   0   0   0   2   5   0   0   0   0   0   0   0   5]
#  [  0   1   1   4   0   0   0   0   1   7   0   0   0   0   0   0   6]
#  [  0   0 152   1   0   0   3   0   4   2   3   0   0   0   1   0   5]
#  [  1   0   2 179  11   0   2   0  14  58   1  16   0   0   1   0  23]
#  [  1   1   1  10  79   0  12   0   2  10   1   1   0   0   1   0  18]
#  [  2   0   7   0   0 212   1   0   0   2   0   0   0   0   0   1  26]
#  [  0   0   0   6   9   0 100   0   0   3   2   3   0   0   3   1   5]
#  [  4   0   0   0   0   0   0 145   0   0   0   0   0   0   0   0   2]
#  [  6   0   4  14   7   1   5   0 209   6   1   2   0   0   3   0   5]
#  [  0   5   5  47   1   0   6   0   9 150   2  11   0   0   3   5  38]
#  [  0   0   4   4   0   0   0   0   1   3   8   0   0   0   0   0   1]
#  [  0   0   0   5   0   0   0   0   2   7   0   6   0   0   1   0   4]
#  [  0   0   0   0   0   1   2   0   0   0   0   0   7   1   0   0   2]
#  [  0   0   0   0   0   0   0   2   0   0   0   1   0  10   0   0   1]
#  [  0   0   0   3   1   0   0   0  10   3   0   0   0   0  14   0   2]
#  [  0   0   0   9   0   0   0   0   2  40   0   2   0   0   0  37   5]
#  [  0   0  10   9   2   5   4   1   6  11   1   2   0   0   0   2 542]]


#              precision    recall  f1-score   support

#           0       0.81      0.82      0.81        72
#           1       0.12      0.05      0.07        20
#           2       0.82      0.89      0.85       171
#           3       0.62      0.58      0.60       308
#           4       0.72      0.58      0.64       137
#           5       0.97      0.84      0.90       251
#           6       0.74      0.76      0.75       132
#           7       0.97      0.96      0.96       151
#           8       0.79      0.79      0.79       263
#           9       0.50      0.53      0.51       282
#          10       0.42      0.38      0.40        21
#          11       0.14      0.24      0.17        25
#          12       1.00      0.54      0.70        13
#          13       0.91      0.71      0.80        14
#          14       0.52      0.42      0.47        33
#          15       0.80      0.39      0.52        95
#          16       0.79      0.91      0.84       595

# avg / total       0.74      0.74      0.74      2583

In [21]:
# >>> FNN Evaluation
# 0.733643050716
# [[ 55   0   1   0   0   1   0   1   9   0   0   0   0   0   0   0   5]
#  [  1   5   0   3   1   0   0   0   1   6   0   0   0   0   0   0   3]
#  [  0   0 153   0   0   1   1   0   7   3   2   0   0   0   0   0   4]
#  [  0   0   1 159  17   0   3   0  17  64   1   5   0   0   3  16  22]
#  [  0   0   2  13  87   0  11   0   1  11   0   1   0   0   1   0  10]
#  [  2   0   6   0   0 227   1   0   0   0   0   0   1   0   0   0  14]
#  [  0   0   0   5  11   0  97   0   1   5   2   3   0   0   2   0   6]
#  [  5   0   0   0   0   0   0 146   0   0   0   0   0   0   0   0   0]
#  [  7   0   3  14   4   2   3   0 210   2   0   0   0   0   8   0  10]
#  [  1   4   4  46   1   1   8   0   7 147   3   9   1   0   6   9  35]
#  [  0   0   6   2   0   0   1   0   0   2   7   0   0   0   0   0   3]
#  [  0   0   0   5   0   0   1   0   3   5   0   4   0   0   1   0   6]
#  [  0   0   0   0   0   1   2   0   0   1   0   0   7   1   0   0   1]
#  [  0   0   0   0   0   0   0   2   0   1   0   0   0  10   0   0   1]
#  [  0   0   0   3   0   0   0   0   7   5   0   0   0   0  15   0   3]
#  [  0   0   0   3   0   0   0   0   3  42   0   1   0   0   0  41   5]
#  [  3   0   5   7   3   6   6   3  12  20   0   0   0   0   2   3 525]]
#              precision    recall  f1-score   support

#           0       0.74      0.76      0.75        72
#           1       0.56      0.25      0.34        20
#           2       0.85      0.89      0.87       171
#           3       0.61      0.52      0.56       308
#           4       0.70      0.64      0.67       137
#           5       0.95      0.90      0.93       251
#           6       0.72      0.73      0.73       132
#           7       0.96      0.97      0.96       151
#           8       0.76      0.80      0.78       263
#           9       0.47      0.52      0.49       282
#          10       0.47      0.33      0.39        21
#          11       0.17      0.16      0.17        25
#          12       0.78      0.54      0.64        13
#          13       0.91      0.71      0.80        14
#          14       0.39      0.45      0.42        33
#          15       0.59      0.43      0.50        95
#          16       0.80      0.88      0.84       595

# avg / total       0.73      0.73      0.73      2583

In [22]:
# hidden node 2: 54.08%
# hidden node 17: 65.69%
# hidden node 30: 66.03%
# hidden node 100: 66.97%

# RNN : parsing tree feature 추가하니까 67.68% 조금 올랐다. 반면, LR은 조금 떨어짐

The proper way to evaluate precision and recall is to use `preds = model.predict(X_test)` and then compute `precision(preds, y_test)` via e.g. a sklearn utility function.

## Conv2

In [24]:
# # fix random seed for reproducibility 
# seed = 7 
# numpy.random.seed(seed) 
# # load the dataset but only keep the top n words, zero the rest 
# top_words = 5000 
# (X_train, y_train), (X_test, y_test) = imdb.load_data(nb_words=top_words) 
# # pad dataset to a maximum review length in words 
# max_words = 50 
# X_train = sequence.pad_sequences(X_train, maxlen=max_words) 
# X_test = sequence.pad_sequences(X_test, maxlen=max_words) 
# print(X_train.shape, y_train.shape)

In [ ]:
# temp_x = [[[3],[2],[1]],[[3],[2],[1]],[[3],[2],[1]],[[3],[2],[1]],[[3],[2],[1]]]
# temp_x = [[[3]],[[1]],[[1]],[[1]],[[1]]]
# temp_x = [[[2],[1]],[[2],[1]],[[2],[1]],[[2],[1]],[[2],[1]]]
# temp_x = numpy.array(temp_x)
# temp_y = [1, 0, 0, 0, 1]
# temp_y = numpy.array(temp_y)
# print(temp_x.shape, temp_y.shape)

In [ ]:
# (X_train).shape

In [ ]:
# # create the model 
# model = Sequential() 
# #model.add(Embedding(top_words, 32, input_length=max_words))
# #model.add(Convolution1D(nb_filter=32, filter_length=3, border_mode='same', activation='relu'))

# #model.add(Convolution1D(64, 3, border_mode='same', input_shape=(3, 1)))
# model.add(Convolution1D(nb_filter=32, filter_length=2, border_mode='same', activation='relu', input_shape=(2, 1)))
 
# model.add(MaxPooling1D(pool_length=2)) 
# model.add(Flatten()) 

# model.add(Dense(250, activation='relu')) 
# model.add(Dense(1, activation='sigmoid')) 
# model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy']) 
# print(model.summary()) 

In [ ]:
# # Fit the model 
# model.fit(temp_x, dummy_y, validation_data=(temp_x, dummy_y), nb_epoch=2, batch_size=128, verbose=1) 

In [ ]:
# # Final evaluation of the model 
# scores = model.evaluate(temp_x, temp_y, verbose=0) 
# print("Accuracy: %.2f%%" % (scores[1]*100))

## Train RNN

* RNN (LSTM) 모델에 대한 engineering도 나중에 해보자.
   - 원래는 validation set와 함께 학습을 통해서 model capacity와 같은 parameter를 결정해야되지만 데이터가 작으면 그냥 하나씩 일일이 해보면서 제일 좋은 parameter를 찾아내는 것도 괜찮은 방법이라 생각한다.
* Bi-directional LSTM은 어떨까?

## Evaluate

In [ ]:
# # summarize performance of the model 
# scores = model.evaluate(X_test_RNN, Y_test_RNN, verbose=0) 
# print("Model Accuracy: %.2f%%" % (scores[1]*100)) 
# # # demonstrate some model predictions 
# # for i in range(20): 
# #     pattern_index = numpy.random.randint(len(dataX)) 
# #     pattern = dataX[pattern_index] 
# #     x = pad_sequences([pattern], maxlen=max_len, dtype='float32') 
# #     x = numpy.reshape(x, (1, max_len, 1)) 
# #     x = x / float(len(alphabet)) 
# #     prediction = model.predict(x, verbose=0) 
# #     index = numpy.argmax(prediction) 
# #     result = int_to_char[index] 
# #     seq_in = [int_to_char[value] for value in pattern] 
# #     print(seq_in, "->", result)

In [ ]:
# statistical
66.97 # all

In [ ]:
## # fix random seed for reproducibility 
# numpy.random.seed(7) 
# # define the raw dataset 
# alphabet = "ABCDEFGHIJKLMNOPQRSTUVWXYZ" 
# # create mapping of characters to integers (0-25) and the reverse 
# char_to_int = dict((c, i) for i, c in enumerate(alphabet)) 
# int_to_char = dict((i, c) for i, c in enumerate(alphabet)) 
# # prepare the dataset of input to output pairs encoded as integers 
# num_inputs = 1000 
# max_len = 3 
# dataX = [] 
# dataY = [] 
# for i in range(num_inputs): 
#     start = numpy.random.randint(len(alphabet)-2) 
#     end = numpy.random.randint(start, min(start+max_len,len(alphabet)-1)) 
#     sequence_in = alphabet[start:end+1] 
#     sequence_out = alphabet[end + 1] 
#     dataX.append([char_to_int[char] for char in sequence_in]) 
#     dataY.append(char_to_int[sequence_out]) 
#     print(sequence_in, '->', sequence_out) 
    
# # convert list of lists to array and pad sequences if needed 
# X = pad_sequences(dataX, maxlen=max_len, dtype='float32') 
# # reshape X to be [samples, time steps, features] 
# X = numpy.reshape(X, (X.shape[0], max_len, 1)) 

# # normalize 
# X = X / float(len(alphabet)) 
# # one hot encode the output variable 
# y = np_utils.to_categorical(dataY) 